In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
import tensorflow as tf
import numpy as np

# Set the seed for random operations. 
# This let our experiments to be reproducible. 
SEED = 1234
tf.random.set_seed(SEED)  

cwd = os.getcwd()

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#here we use original size images

#Creation of the dataset for Haricot model

#!mkdir -p '/content/drive/My Drive/Kaggle2/Training/Bipbip/Haricot/Images/img'
#!mkdir -p '/content/drive/My Drive/Kaggle2/Training/Bipbip/Haricot/Masks/img'


#!cp '/content/drive/My Drive/Kaggle2/Development_Dataset/Training/Bipbip/Haricot/Images/* '/content/drive/My Drive/Kaggle2/Training/Bipbip/Haricot/Images/img'
#!cp /content/drive/My Drive/Kaggle2/Development_Dataset/Training/Bipbip/Haricot/Masks/* '/content/drive/My Drive/Kaggle2/Training/Bipbip/Haricot/Masks/img'


#Creation of the dataset for Mais model
!mkdir -p '/content/drive/My Drive/Kaggle2/Training/Bipbip/Mais/Images/img'
!mkdir -p '/content/drive/My Drive/Kaggle2/Training/Bipbip/Mais/Masks/img'



!cp /content/drive/My Drive/Kaggle2/Development_Dataset/Training/Bipbip/Mais/Images/* '/content/drive/My Drive/Kaggle2/Training/Bipbip/Mais/Images/img'
!cp /content/drive/My Drive/Kaggle2/Development_Dataset/Training/Bipbip/Mais/Masks/* '/content/drive/My Drive/Kaggle2/Training/Bipbip/Mais/Masks/img'



In [ ]:
# ImageDataGenerator
# ------------------

from tensorflow.keras.preprocessing.image import ImageDataGenerator

apply_data_augmentation = True

# Create training ImageDataGenerator object
# We need two different generators for images and corresponding masks
if apply_data_augmentation:
    train_img_data_gen = ImageDataGenerator(rotation_range=10,
                                      width_shift_range=10,
                                      height_shift_range=10,
                                      zoom_range=0.3,
                                      horizontal_flip=True,
                                      vertical_flip=True,
                                      fill_mode='reflect',
                                      rescale=1./255,
                                      validation_split=0.2)
    train_mask_data_gen = ImageDataGenerator(rotation_range=10,
                                       width_shift_range=10,
                                       height_shift_range=10,
                                       zoom_range=0.3,
                                       horizontal_flip=True,
                                       vertical_flip=True,
                                       fill_mode='reflect',
                                       validation_split=0.2)
else:
    train_img_data_gen = ImageDataGenerator(rescale=1./255,validation_split=0.2)
    train_mask_data_gen = ImageDataGenerator(validation_split=0.2)

# Create validation ImageDataGenerator objects
valid_img_data_gen = ImageDataGenerator(rescale=1./255,validation_split=0.2)
valid_mask_data_gen = ImageDataGenerator(validation_split=0.2)

# Create generators to read images from dataset directory
# -------------------------------------------------------
#dataset_dir = os.path.join(cwd, 'Development_Dataset')
dataset_dir = '/content/drive/My Drive/Kaggle2/Training/Bipbip'
# Batch size
bs = 4

# img shape bipbip
img_h = 1536
img_w = 2048
num_classes=3

# Training
# Two different generators for images and masks but with same seed
training_dir = os.path.join(dataset_dir, 'Mais')  #put 'Haricot' for the other model
train_img_gen = train_img_data_gen.flow_from_directory(os.path.join(training_dir, 'Images'),
                                                       target_size=(img_h, img_w),
                                                       batch_size=bs,
                                                       class_mode=None, 
                                                       shuffle=True,
                                                       interpolation='bilinear',
                                                       seed=SEED,
                                                       subset='training')  
train_mask_gen = train_mask_data_gen.flow_from_directory(os.path.join(training_dir, 'Masks'),
                                                         target_size=(img_h, img_w),
                                                         batch_size=bs,
                                                         class_mode=None,
                                                         shuffle=True,
                                                         interpolation='nearest',
                                                         seed=SEED,
                                                         subset='training')
train_gen = zip(train_img_gen, train_mask_gen)

# Validation
valid_img_gen = valid_img_data_gen.flow_from_directory(os.path.join(training_dir, 'Images'),
                                                       target_size=(img_h, img_w),
                                                       batch_size=bs, 
                                                       class_mode=None, 
                                                       shuffle=False,
                                                       interpolation='bilinear',
                                                       seed=SEED,
                                                       subset='validation')
valid_mask_gen = valid_mask_data_gen.flow_from_directory(os.path.join(training_dir, 'Masks'),
                                                         target_size=(img_h, img_w),
                                                         batch_size=bs, 
                                                         class_mode=None,
                                                         shuffle=False,
                                                         interpolation='nearest',
                                                         seed=SEED,
                                                         subset='validation')
valid_gen = zip(valid_img_gen, valid_mask_gen)

In [ ]:
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None,img_h, img_w, 3], [None,img_h, img_w, 3]))


def prepare_target(x_, y_):
  # Use RGB dictionary in 'RGBtoTarget.txt' to convert RGB to target labels 256 x 256 x 1
  #[254, 124, 18] = 0 done implicitly with the first cast
  #[255, 255, 255]= 1 done with the first cast
  #[216, 67, 82]= 2 done with the second cast that is multiplied by 2
    output=tf.cast(tf.reduce_all(y_== [255, 255, 255], axis=-1, keepdims=True), tf.float32)
    output=output+2*tf.cast(tf.reduce_all(y_== [216, 67, 82], axis=-1, keepdims=True), tf.float32)
    return x_, output

train_dataset = train_dataset.map(prepare_target)


train_dataset = train_dataset.repeat()

valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None,img_h, img_w, 3], [None,img_h, img_w,3]))
valid_dataset = valid_dataset.map(prepare_target)


valid_dataset = valid_dataset.repeat()

In [ ]:
# Let's test data generator for a check
# -------------------------
import time
from matplotlib import cm
import matplotlib.pyplot as plt

%matplotlib inline

# Assign a color to each class
evenly_spaced_interval = np.linspace(0, 1, num_classes-1)
colors = [cm.rainbow(x) for x in evenly_spaced_interval]

iterator = iter(train_dataset)

fig, ax = plt.subplots(1, 2)

augmented_img, target = next(iterator)
augmented_img = np.array(augmented_img[0])   # First element
augmented_img = augmented_img *255           # denormalize

target = np.array(target[0, ..., 0])         # First element (squeezing channel dimension)
print(np.unique(target))

target_img = np.zeros([target.shape[0], target.shape[1], 3])

target_img[np.where(target == 0)] = [0, 0, 0]
for i in range(1, num_classes):
  target_img[np.where(target == i)] = np.array(colors[i-1])[:3] * 255

ax[0].imshow(np.uint8(augmented_img))
ax[1].imshow(np.uint8(target_img))

plt.show()

In [ ]:
# Create Model
# ------------

def create_model(depth, start_f, dynamic_input_shape):

    model = tf.keras.Sequential()
    
    # Encoder
    # -------
    for i in range(depth):
        
        if i == 0:
            if dynamic_input_shape:
                input_shape = [None, None, 3]
            else:
                input_shape = [img_h, img_w, 3]
        else:
            input_shape=[None]
        
        model.add(tf.keras.layers.Conv2D(filters=start_f, 
                                         kernel_size=(3, 3),
                                         strides=(1, 1),
                                         padding='same',
                                         input_shape=input_shape))
        model.add(tf.keras.layers.ReLU())
        model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

        start_f *= 2

    # Bottleneck
    model.add(tf.keras.layers.Conv2D(filters=start_f, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    model.add(tf.keras.layers.ReLU())
    
    start_f = start_f // 2
        
    # Decoder
    # -------
    for i in range(depth):
        model.add(tf.keras.layers.UpSampling2D(2, interpolation='bilinear'))
        model.add(tf.keras.layers.Conv2D(filters=start_f,
                                         kernel_size=(3, 3),
                                         strides=(1, 1),
                                         padding='same'))
        model.add(tf.keras.layers.ReLU())

        start_f = start_f // 2

    # Prediction Layer
    # ----------------
    model.add(tf.keras.layers.Conv2D(filters=num_classes,
                                     kernel_size=(1, 1),
                                     strides=(1, 1),
                                     padding='same',
                                     activation='softmax'))
    
    return model

In [ ]:
model= create_model(depth=6, 
                    start_f=16,
                    dynamic_input_shape=True)
# Visualize created model as a table
model.summary()

In [ ]:
# Optimization params
# -------------------

# Loss
# Sparse Categorical Crossentropy to use integers (mask) instead of one-hot encoded labels
loss = tf.keras.losses.SparseCategoricalCrossentropy() 
# learning rate
lr = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Here we define the intersection over union for each class in the batch.
# Then we compute the final iou as the mean over classes
def meanIoU(y_true, y_pred):
    # get predicted class from softmax
    y_pred = tf.expand_dims(tf.argmax(y_pred, -1), -1)

    per_class_iou = []

    for i in range(1,3): # exclude the background class 0
      # Get prediction and target related to only a single class (i)
      class_pred = tf.cast(tf.where(y_pred == i, 1, 0), tf.float32)
      class_true = tf.cast(tf.where(y_true == i, 1, 0), tf.float32)
      intersection = tf.reduce_sum(class_true * class_pred)
      union = tf.reduce_sum(class_true) + tf.reduce_sum(class_pred) - intersection
    
      iou = (intersection + 1e-7) / (union + 1e-7)
      per_class_iou.append(iou)

    return tf.reduce_mean(per_class_iou)

# Validation metrics
# ------------------
metrics = ['accuracy', meanIoU]
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
#callbacks for training  
callbacks = []

# Early Stopping
# --------------
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    callbacks.append(es_callback)

In [ ]:
#training
model.fit(x=train_dataset,
          epochs=50,  
          steps_per_epoch=len(train_img_gen ),
          validation_data=valid_dataset,
          validation_steps=len(valid_img_gen ), 
          callbacks=callbacks)

In [ ]:
#saving the model for final ensembling
model.save('/content/drive/My Drive/Kaggle2/SavedModel/basic_mais.h5') 


In [ ]:
#loading models for predictions

def meanIoU(y_true, y_pred):
    # get predicted class from softmax
    y_pred = tf.expand_dims(tf.argmax(y_pred, -1), -1)

    per_class_iou = []

    for i in range(1,3): # exclude the background class 0
      # Get prediction and target related to only a single class (i)
      class_pred = tf.cast(tf.where(y_pred == i, 1, 0), tf.float32)
      class_true = tf.cast(tf.where(y_true == i, 1, 0), tf.float32)
      intersection = tf.reduce_sum(class_true * class_pred)
      union = tf.reduce_sum(class_true) + tf.reduce_sum(class_pred) - intersection
    
      iou = (intersection + 1e-7) / (union + 1e-7)
      per_class_iou.append(iou)

    return tf.reduce_mean(per_class_iou)


model_haricot=tf.keras.models.load_model('/content/drive/My Drive/Kaggle2/SavedModel/basic_haricot.h5',custom_objects={'meanIoU':meanIoU})

model_mais=tf.keras.models.load_model('/content/drive/My Drive/Kaggle2/SavedModel/basic_mais.h5',custom_objects={'meanIoU':meanIoU})

In [ ]:
# prepare submission

from PIL import Image
def rle_encode(img):
    '''
    img: numpy array, 1 - foreground, 0 - background
    Returns run length as string formatted
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

submission_dict = {}
groups=['Bipbip','Pead','Roseau','Weedelec']
hm=['Haricot','Mais']
for g in groups:
  for h in hm:
      final_path = '/content/drive/My Drive/Kaggle2/Test_Dev/'+ str(g) + '/'+ str(h) +'/Images'
      image_filenames = next(os.walk(final_path))[2]   
      for img_name in image_filenames:
          img=Image.open(final_path+'/'+img_name).convert('RGB')
          img_array=np.array(img)/255
          if h=='Mais':
              out_sigmoid = model_mais.predict(x=tf.expand_dims(img_array, 0))
          else:
              out_sigmoid = model_haricot.predict(x=tf.expand_dims(img_array, 0))
          predicted_class = tf.argmax(out_sigmoid, -1)
          predicted_class = predicted_class[0, ...]
          mask_arr = np.array(predicted_class)
          # RLE encoding
          # crop
          rle_encoded_crop = rle_encode(mask_arr == 1)
          # weed
          rle_encoded_weed = rle_encode(mask_arr == 2)
          img_name_without_extension=img_name[:-4]
          submission_dict[img_name_without_extension] = {}
          submission_dict[img_name_without_extension]['shape'] = mask_arr.shape
          submission_dict[img_name_without_extension]['team'] = g
          submission_dict[img_name_without_extension]['crop'] = h
          submission_dict[img_name_without_extension]['segmentation'] = {}
          submission_dict[img_name_without_extension]['segmentation']['crop'] = rle_encoded_crop
          submission_dict[img_name_without_extension]['segmentation']['weed'] = rle_encoded_weed



In [ ]:
# save submission
import json
with open('/content/drive/My Drive/Kaggle2/finale_basic_submission.json', 'w') as f:
  json.dump(submission_dict, f)